In [1]:
import pandas as pd
import numpy as np
import keras
from keras.utils.data_utils import get_file
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from os.path import join
import multiprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from scipy.stats import skew, kurtosis, tvar, entropy, pearsonr
import keras.backend.tensorflow_backend as K

%matplotlib inline

Using TensorFlow backend.


In [2]:
# DATA_HOME = '../lfw-aligned-cropped/'

DATA_HOME = '../lfw-aligned-with-32-margin-resized/'

In [3]:
PAIRS_PATH = '../pairs.txt'

In [4]:
raw_dataset = pd.read_csv(PAIRS_PATH, nrows=None)
raw_dataset.columns = ['data']

In [5]:
raw_dataset.shape

(6000, 1)

In [6]:
raw_dataset = raw_dataset

In [7]:
def split_raw_data(row):
    fields = str(row[0]).split('\t')
    is_the_same = 0
    path1, path2 = '{}/{}_{:04d}.png', '{}/{}_{:04d}.png'
    if len(fields) == 3:
        is_the_same = 1
        path1 = path1.format(fields[0], fields[0], int(fields[1]))
        path2 = path2.format(fields[0], fields[0], int(fields[2]))
    elif len(fields) == 4:
        path1 = path1.format(fields[0], fields[0], int(fields[1]))
        path2 = path2.format(fields[2], fields[2], int(fields[3]))
    else:
        raise(Exception('invalid fields'))
    
    return pd.Series([path1, path2, is_the_same], ['path1', 'path2', 'is_the_same']) 

In [8]:
dataset = raw_dataset.apply(split_raw_data, axis=1)

In [9]:
dataset.head()

,path1,path2,is_the_same
0,Abel_Pacheco/Abel_Pacheco_0001.png,Abel_Pacheco/Abel_Pacheco_0004.png,1
1,Akhmed_Zakayev/Akhmed_Zakayev_0001.png,Akhmed_Zakayev/Akhmed_Zakayev_0003.png,1
2,Akhmed_Zakayev/Akhmed_Zakayev_0002.png,Akhmed_Zakayev/Akhmed_Zakayev_0003.png,1
3,Amber_Tamblyn/Amber_Tamblyn_0001.png,Amber_Tamblyn/Amber_Tamblyn_0002.png,1
4,Anders_Fogh_Rasmussen/Anders_Fogh_Rasmussen_00...,Anders_Fogh_Rasmussen/Anders_Fogh_Rasmussen_00...,1


In [10]:
dataset.shape

(6000, 3)

In [11]:
def path2ImgVec(path):
    x = img_to_array(load_img(join(DATA_HOME, path)))
    return x.reshape((1,) + x.shape)

In [12]:
img_paths = list(set(dataset.path1.tolist()+dataset.path2.tolist()))
len(img_paths)

7701

In [13]:
pool = multiprocessing.Pool(8)
results = pool.map(path2ImgVec, img_paths)
pool.close()
pool.join()

In [14]:
img_vecs = np.vstack(results)
img_vecs.shape

(7701, 55, 47, 3)

In [15]:
input_shape = img_vecs.shape[1:]
img_vecs.shape

(7701, 55, 47, 3)

In [16]:
from keras.models import load_model

# model = load_model('../models/facescrub-faceonly-simple-cnn.model.h5')
# model = load_model('../models/webface-simple-cnn.3348.model.h5')
model = load_model('../models/faceonly-simple-cnn.aligned.margin32.model.h5')

In [17]:
from keras import backend as K
inp = model.input
# outputs = [layer.output for layer in model.layers]
outputs = [model.layers[-2].output]
functor = K.function([inp] + [K.learning_phase()], outputs)

In [18]:
deepids = functor([img_vecs, 1.])[0]

In [19]:
deepids.shape

(7701, 160)

In [20]:
img_deepid_mem = {k:v for k, v in zip(img_paths, deepids)}

In [21]:
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis

COL_NAMES = [
    "z_cosine",
#     'z_cityblock',
#     'z_jaccard',
#     "z_canberra",
#     "z_euclidean",
#     "z_minkowski",
#     "z_braycurtis",
    'z_skew1',
    'z_skew2',
    'z_kurtosis1',
    'z_kurtosis2',
    'z_tvar1',
    'z_tvar2',
] + ['is_the_same']

def to_deepid_features(row):
    vec1 = img_deepid_mem[row['path1']]
    vec2 = img_deepid_mem[row['path2']]

    feats = [
        cosine(vec1, vec2),
#         cityblock(vec1, vec2),
#         jaccard(vec1, vec2),
#         canberra(vec1, vec2),
#         euclidean(vec1, vec2),
#         minkowski(vec1, vec2, 3),
#         braycurtis(vec1, vec2),
        skew(vec1),
        skew(vec2),
        kurtosis(vec1),
        kurtosis(vec2),
        tvar(vec1),
        tvar(vec2),
    ]
    
    return pd.Series(feats + [row['is_the_same']], COL_NAMES) 

In [22]:
feat_dataset = dataset.apply(to_deepid_features, axis=1)

In [23]:
feat_dataset.head()

,z_cosine,z_skew1,z_skew2,z_kurtosis1,z_kurtosis2,z_tvar1,z_tvar2,is_the_same
0,0.551305,1.243931,1.215489,0.722285,0.474363,1.216830,1.966980,1.0
1,0.336454,1.127347,1.301833,0.191542,0.985561,1.356894,1.712710,1.0
2,0.515355,1.027075,1.301833,0.158675,0.985561,1.854493,1.712710,1.0
3,0.402206,1.057403,1.374781,0.318135,1.363059,1.337263,1.823520,1.0
4,0.445874,0.882318,1.406964,-0.017139,1.894684,1.180420,1.705738,1.0


In [24]:
col = [c for c in feat_dataset.columns if c[0]=='z']

X = feat_dataset[col].as_matrix()
y = feat_dataset['is_the_same'].as_matrix()

In [25]:
X[0]

array([ 0.55130534,  1.24393094,  1.21548903,  0.72228494,  0.4743627 ,
        1.21682983,  1.96698009])

In [26]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=42)

In [27]:
import xgboost as xgb
params = {}
params["objective"] = "binary:logistic"
params['eval_metric'] = ['logloss', 'error']
# params["eta"] = 0.02
# params["subsample"] = 0.7
# params["min_child_weight"] = 1
# params["colsample_bytree"] = 0.7
params["max_depth"] = 4
# params["silent"] = 1
params["seed"] = 1632

d_train = xgb.DMatrix(X_train, label=y_train)
d_valid = xgb.DMatrix(X_valid, label=y_valid)
watchlist = [(d_train, 'train'), (d_valid, 'valid')]
bst = xgb.train(params, d_train, 500, watchlist, early_stopping_rounds=50, verbose_eval=100)

[0]	train-logloss:0.601939	train-error:0.23537	valid-logloss:0.600242	valid-error:0.251667
Multiple eval metrics have been passed: 'valid-error' will be used for early stopping.

Will train until valid-error hasn't improved in 50 rounds.
Stopping. Best iteration:
[24]	train-logloss:0.42641	train-error:0.198704	valid-logloss:0.462621	valid-error:0.216667



In [28]:
d_all = xgb.DMatrix(X, label=y)

xgb.cv(params, d_all, 500, nfold=10, metrics=['error'], shuffle=False, early_stopping_rounds=20, seed=42)

,test-error-mean,test-error-std,train-error-mean,train-error-std
0,0.251167,0.016667,0.235797,0.003160
1,0.248667,0.021932,0.231759,0.002700
2,0.249500,0.020522,0.231185,0.002415
3,0.250667,0.021733,0.230092,0.002651
4,0.245166,0.020241,0.229056,0.002492
